# Protein-ligand Docking tutorial using BioExcel Building Blocks (biobb)
### -- *Fpocket Version* --

***
This tutorial aims to illustrate the process of **protein-ligand docking**, step by step, using the **BioExcel Building Blocks library (biobb)**. The particular example used is the **Mitogen-activated protein kinase 14** (p38-α) protein (PDB code [3HEC](https://www.rcsb.org/structure/3HEC)), a well-known **Protein Kinase enzyme**, 
 in complex with the FDA-approved **Imatinib**, (PDB Ligand code [STI](https://www.rcsb.org/ligand/STI), DrugBank Ligand Code [DB00619](https://go.drugbank.com/drugs/DB00619)), a small molecule **kinase inhibitor** used to treat certain types of **cancer**. 
 
The tutorial will guide you through the process of identifying the **active site cavity** (pocket) without previous knowledge, and the final prediction of the **protein-ligand complex**. 

Please note that **docking algorithms**, and in particular, **AutoDock Vina** program used in this tutorial, are **non-deterministic**. That means that results obtained when running the workflow **could be diferent** from the ones we obtained during the writing of this tutorial (see [AutoDock Vina manual](http://vina.scripps.edu/manual.html)). We invite you to try the docking process several times to verify this behaviour. 
***


<div style="background:#b5e0dd; padding: 15px;"><strong>Important:</strong> it is recommended to execute this tutorial step by step (not as a single workflow execution, <strong><em>Run All</em></strong> mode), as it has interactive selections.</div>

## Settings

### Biobb modules used

 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
 - [biobb_structure_utils](https://github.com/bioexcel/biobb_structure_utils): Tools to modify or extract information from a PDB structure file.
 - [biobb_chemistry](https://github.com/bioexcel/biobb_chemistry): Tools to perform chemoinformatics processes.
 - [biobb_vs](https://github.com/bioexcel/biobb_vs): Tools to perform virtual screening studies.
 
### Auxiliar libraries used

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.

### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_virtual-screening.git
cd biobb_wf_virtual_screening
conda env create -f conda_env/environment.yml
conda activate biobb_VS_tutorial
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-nbextension enable --py --user nglview
jupyter-notebook biobb_wf_virtual_screening/notebooks/fpocket/wf_vs_fpocket.ipynb
  ``` 

***
## Pipeline steps
 1. [Input Parameters](#input)
 2. [Fetching PDB Structure](#fetch)
 3. [Extract Protein Structure](#extractProtein)
 4. [Computing Protein Cavities (fpocket)](#fpocket)
 5. [Filtering Protein Cavities (fpocket output)](#fpocketFilter)
 6. [Extract Pocket Cavity ](#fpocketSelect)
 7. [Generating Cavity Box ](#cavityBox)
 8. [Downloading Small Molecule](#downloadSmallMolecule)
 9. [Converting Small Molecule](#sdf2pdb)
 10. [Preparing Small Molecule (ligand) for Docking](#ligand_pdb2pdbqt)
 11. [Preparing Target Protein for Docking](#protein_pdb2pdbqt)
 12. [Running the Docking](#docking)
 13. [Extract a Docking Pose](#extractPose)
 14. [Converting Ligand Pose to PDB format](#pdbqt2pdb)
 15. [Superposing Ligand Pose to the Target Protein Structure](#catPdb)
 16. [Comparing final result with experimental structure](#viewFinal)
 17. [Questions & Comments](#questions)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***


<a id="input"></a>
## Input parameters
**Input parameters** needed:

 - **pdb_code**: PDB code of the experimental complex structure (if exists).<br>
In this particular example, the **p38α** structure in complex with the **Imatinib drug** was experimentally solved and deposited in the **PDB database** under the **3HEC** PDB code. The protein structure from this PDB file will be used as a **target protein** for the **docking process**, after stripping the **small molecule**. An **APO structure**, or any other structure from the **p38α** [cluster 100](https://www.rcsb.org/search?request=%7B%22query%22%3A%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22sequence%22%2C%22parameters%22%3A%7B%22target%22%3A%22pdb_protein_sequence%22%2C%22value%22%3A%22RPTFYRQELNKTIWEVPERYQNLSPVGSGAYGSVCAAFDTKTGLRVAVKKLSRPFQSIIHAKRTYRELRLLKHMKHENVIGLLDVFTPARSLEEFNDVYLVTHLMGADLNNIVKCQKLTDDHVQFLIYQILRGLKYIHSADIIHRDLKPSNLAVNEDCELKILDFGLARHTDDEMTGYVATRWYRAPEIMLNWMHYNQTVDIWSVGCIMAELLTGRTLFPGTDHIDQLKLILRLVGTPGAELLKKISSESARNYIQSLTQMPKMNFANVFIGANPLAVDLLEKMLVLDSDKRITAAQALAHAYFAQYHDPDDEPVADPYDQSFESRDLLIDEWKSLTYDEVISFVPPP%22%2C%22identity_cutoff%22%3A1%2C%22evalue_cutoff%22%3A0.1%7D%2C%22node_id%22%3A0%7D%2C%22return_type%22%3A%22polymer_entity%22%2C%22request_options%22%3A%7B%22pager%22%3A%7B%22start%22%3A0%2C%22rows%22%3A25%7D%2C%22scoring_strategy%22%3A%22combined%22%2C%22sort%22%3A%5B%7B%22sort_by%22%3A%22score%22%2C%22direction%22%3A%22desc%22%7D%5D%7D%2C%22request_info%22%3A%7B%22src%22%3A%22ui%22%2C%22query_id%22%3A%22bea5861f8b38a9e25a3e626b39d6bcbf%22%7D%7D) (sharing a 100% of sequence similarity with the **p38α** structure) could also be used as a **target protein**. This structure of the **protein-ligand complex** will be also used in the last step of the tutorial to check **how close** the resulting **docking pose** is from the known **experimental structure**. 
 -----
 - **ligandCode**: Ligand PDB code (3-letter code) for the small molecule (e.g. STI).<br>
In this particular example, the small molecule chosen for the tutorial is the FDA-approved drug **Imatinib** (PDB Code STI), a type of cancer growth blocker, used in [diferent types of leukemia](https://go.drugbank.com/drugs/DB00619).
 -----
 - **pockets_dir**: Name of a folder to write temporary files.

In [1]:
import nglview
import ipywidgets

pdb_code = "3HEC"         # P38 + Imatinib

ligand_code = "STI"       # Imatinib

pockets_dir = "pockets"

<a id="fetch"></a>
***
## Fetching PDB structure
Downloading **PDB structure** with the **protein molecule** from the PDBe database.<br>
Alternatively, a **PDB file** can be used as starting structure. <br>
***
**Building Blocks** used:
 - [Pdb](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.pdb) from **biobb_io.api.pdb**
***

In [2]:
from biobb_io.api.pdb import pdb

download_pdb = "download.pdb"
prop = {
  "pdb_code": pdb_code,
  "filter": ["ATOM", "HETATM"]
}

pdb(output_pdb_path=download_pdb,
    properties=prop)

2021-05-17 15:44:24,396 [MainThread  ] [INFO ]  Downloading: 3hec from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb3hec.ent
/anaconda3/envs/biobb_VS_tutorial/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ebi.ac.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
2021-05-17 15:44:24,864 [MainThread  ] [INFO ]  Writting pdb to: download.pdb
2021-05-17 15:44:24,870 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'HETATM']


0

<a id="vis3D"></a>
### Visualizing 3D structure
Visualizing the downloaded/given **PDB structure** using **NGL**.<br><br>
Note (and try to identify) the **Imatinib small molecule (STI)** and the **detergent (β-octyl glucoside) (BOG)** used in the experimental reservoir solution to obtain the crystal.

In [27]:
view = nglview.show_structure_file(download_pdb, default=True)
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])

view.render_image()
view.download_image(filename='ngl1.png')
view

NGLWidget()

<img src='ngl1.png'></img>

<a id="extractProtein"></a>
***
## Extract Protein Structure
Extract **protein structure** from the **downloaded PDB file**. Removing **any extra molecule** (ligands, ions, water molecules). <br><br>
The **protein structure** will be used as a **target** in the **protein-ligand docking process**. 
***
**Building Blocks** used:
 - [extract_protein](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.extract_protein) from **biobb_structure_utils.utils.extract_protein**
***

In [4]:
from biobb_structure_utils.utils.extract_protein import extract_protein

pdb_protein = "pdb_protein.pdb"

extract_protein(input_structure_path=download_pdb,
             output_protein_path = pdb_protein)

2021-05-17 15:44:29,192 [MainThread  ] [INFO ]  Creating b33b03fe-5761-4996-beca-971e0ea316f5 temporary folder
2021-05-17 15:44:29,956 [MainThread  ] [INFO ]  check_structure -i /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_virtual-screening/biobb_wf_virtual-screening/notebooks/fpocket/download.pdb -o pdb_protein.pdb --force_save --non_interactive command_list --list b33b03fe-5761-4996-beca-971e0ea316f5/extract_prot.lst

2021-05-17 15:44:29,958 [MainThread  ] [INFO ]  Exit code 0

2021-05-17 15:44:29,959 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.7.2                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-20                  =

Structure /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_virtual-screening/biobb_wf_virtual-screening/notebooks/fpocket/download.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution: None A

 Num. mo

0

<a id="vis3D"></a>
### Visualizing 3D structure
Visualizing the downloaded/given **PDB structure** using **NGL**.<br><br>
Note that the **small molecules** included in the original structure are now gone. The new structure only contains the **protein molecule**, which will be used as a **target** for the **protein-ligand docking**. 

In [28]:
view = nglview.show_structure_file(pdb_protein, default=False)
view.add_representation(repr_type='cartoon', 
                        selection='not het',
                       colorScheme = 'atomindex')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])

view.render_image()
view.download_image(filename='ngl2.png')
view

NGLWidget()

<img src='ngl2.png'></img>

<a id="fpocket"></a>
***
## Computing Protein Cavities (fpocket)
Computing the **protein cavities** (pockets) using the well-known [**fpocket**](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-10-168) tool.<br>
These **cavities** will be then used in the **docking procedure** to try to find the **best region of the protein surface** where the small molecule can **bind**. <br><br>
Although in this particular example we already know the **binding site** region, as we started from a **protein-ligand complex** structure where the ligand was located in the same **binding site** as **Imatinib** is binding, this is not always the case. In the cases where we do not know these regions, **fpocket** will help us identifying the possible **binding sites** of our **target protein**.<br>

**fpocket** input parameters, such as **minimum** and **maximum radius** (in Angstroms) the alpha spheres might have in a **binding pocket** can be adjusted (min_radius, max_radius) . Parameters used in this particular example are 3Å for the **minimum radius** and 6Å for the **maximum radius**. The **minimum number of alpha spheres** a pocket must contain in order to figure in the results is also adjusted to 35. See the [fpocket manual](http://fpocket.sourceforge.net/manual_fpocket2.pdf) for more information.<br> 
<br>
***
**Building Blocks** used:
 - [fpocket](https://biobb-vs.readthedocs.io/en/latest/fpocket.html#module-fpocket.fpocket) from **biobb_vs.fpocket.fpocket**
***

In [6]:
from biobb_vs.fpocket.fpocket import fpocket

fpocket_all_pockets = "fpocket_all_pockets.zip"
fpocket_summary = "fpocket_summary.json"
prop = {
    "min_radius": 3,
    "max_radius": 6,
    "num_spheres": 35
}

fpocket(input_pdb_path=pdb_protein,
        output_pockets_zip = fpocket_all_pockets,
        output_summary=fpocket_summary,
        properties=prop)

2021-05-17 15:44:34,300 [MainThread  ] [INFO ]  Creating 1c32ea8c-d34c-4f25-8d71-99263c08a6b8 temporary folder
2021-05-17 15:44:34,302 [MainThread  ] [INFO ]  Executing fpocket
2021-05-17 15:44:34,303 [MainThread  ] [INFO ]  Not using any container
2021-05-17 15:44:35,866 [MainThread  ] [INFO ]  fpocket -f 1c32ea8c-d34c-4f25-8d71-99263c08a6b8/input.pdb -m 3 -M 6 -i 35

2021-05-17 15:44:35,867 [MainThread  ] [INFO ]  Exit code 0

2021-05-17 15:44:35,867 [MainThread  ] [INFO ]  ***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 

2021-05-17 15:44:35,871 [MainThread  ] [INFO ]  14 pockets found
2021-05-17 15:44:35,872 [MainThread  ] [INFO ]  Sorting output data by druggability_score
2021-05-17 15:44:35,877 [MainThread  ] [INFO ]  Adding:
2021-05-17 15:44:35,878 [MainThread  ] [INFO ]  ['1c32ea8c-d34c-4f25-8d71-99263c08a6b8/input_out/pockets/pocket10_atm.pdb', '1c32ea8c-d34c-4f25-8d71-99263c08a6b8/input_out/pockets/pocket10_vert.pqr', '1c32ea8c-d34c-4f25-8d71-99263c08a6b8/i

0

<a id="checkJson"></a>
### Checking fpocket output (json)
Checking the **fpocket** output from the **json file**. Every **pocket** has a separated entry in the json output, with information such as: **score, druggability score, volume, hydrophobicity, polarity or flexibility**. 

In [7]:
import json

with open(fpocket_summary, 'r') as json_file:
    data = json.load(json_file)
    print(json.dumps(data, indent=4))

{
    "pocket1": {
        "score": 0.341,
        "druggability_score": 0.876,
        "number_of_alpha_spheres": 227,
        "total_sasa": 357.1,
        "polar_sasa": 93.837,
        "apolar_sasa": 263.263,
        "volume": 1486.347,
        "mean_local_hydrophobic_density": 69.241,
        "mean_alpha_sphere_radius": 3.576,
        "mean_alp_sph_solvent_access": 0.445,
        "apolar_alpha_sphere_proportion": 0.731,
        "hydrophobicity_score": 33.129,
        "volume_score": 4.258,
        "polarity_score": 17,
        "charge_score": 0,
        "proportion_of_polar_atoms": 30.328,
        "alpha_sphere_density": 8.901,
        "cent_of_mass_alpha_sphere_max_dist": 24.197,
        "flexibility": 0.62
    },
    "pocket14": {
        "score": -0.129,
        "druggability_score": 0.041,
        "number_of_alpha_spheres": 61,
        "total_sasa": 188.671,
        "polar_sasa": 61.87,
        "apolar_sasa": 126.801,
        "volume": 539.674,
        "mean_local_hydrophobic_de

<a id="fpocketFilter"></a>
***
## Filtering Protein Cavities (fpocket output)
Filtering the **protein cavities** (pockets) identified by **fpocket**.<br>
In this particular example, the biggest **cavities**, with a **volume** between 800 and 2000 ($Å^{3}$), big enough volume to fit the input small molecule, are selected. <br>

***
**Building Blocks** used:
 - [fpocket_filter](https://biobb-vs.readthedocs.io/en/latest/fpocket.html#module-fpocket.fpocket_filter) from **biobb_vs.fpocket.fpocket_filter**
***

In [8]:
from biobb_vs.fpocket.fpocket_filter import fpocket_filter

fpocket_filter_pockets = "fpocket_filter_pockets.zip"
prop = {
    "volume": [800, 2000]
}

fpocket_filter(input_pockets_zip=fpocket_all_pockets,
                input_summary = fpocket_summary,
                output_filter_pockets_zip=fpocket_filter_pockets,
                properties=prop)

2021-05-17 15:44:40,770 [MainThread  ] [INFO ]  Performing a search under the next parameters: volume: [800, 2000]
2021-05-17 15:44:40,772 [MainThread  ] [INFO ]  Found 2 matches:
**********
pocket1
**********
score: 0.341
druggability_score: 0.876
volume: 1486.347

**********
pocket6
**********
score: 0.064
druggability_score: 0.028
volume: 1898.144

2021-05-17 15:44:40,774 [MainThread  ] [INFO ]  Creating 061d0905-ffb1-4c88-8ba1-bc67ed541a5e temporary folder
2021-05-17 15:44:40,782 [MainThread  ] [INFO ]  Extracting: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_virtual-screening/biobb_wf_virtual-screening/notebooks/fpocket/fpocket_all_pockets.zip
2021-05-17 15:44:40,782 [MainThread  ] [INFO ]  to:
2021-05-17 15:44:40,783 [MainThread  ] [INFO ]  ['061d0905-ffb1-4c88-8ba1-bc67ed541a5e/pocket10_atm.pdb', '061d0905-ffb1-4c88-8ba1-bc67ed541a5e/pocket10_vert.pqr', '061d0905-ffb1-4c88-8ba1-bc67ed541a5e/pocket11_atm.pdb', '061d0905-ffb1-4c88-8ba1-bc67ed541a5e/pocket11_vert.pqr', '0

0

<a id="extractPockets"></a>
### Extract selected pockets (cavities)
Extract the selected **pockets** (cavities) from the filtered list (zip file, fpocket_filter_pockets).<br>
Writing the information in the ***pockets_dir*** folder.<br>
Also saving the list of **PDB files** (protein residues forming the pocket) and **PQR files** (cavity, pocket), to be used in following **visualization step**. 

In [9]:
import os
import shutil

from pathlib import Path, PurePath
import zipfile

if Path(pockets_dir).exists(): shutil.rmtree(pockets_dir) 
os.mkdir(pockets_dir)

with zipfile.ZipFile(fpocket_filter_pockets, 'r') as zip_ref:
    zip_ref.extractall(pockets_dir)

path_pockets = [str(i) for i in Path(pockets_dir).iterdir()]
path_pockets_pdb = [str(i) for i in Path(pockets_dir).iterdir() if PurePath(i).suffix == '.pdb']
path_pockets_pqr = [str(i) for i in Path(pockets_dir).iterdir() if PurePath(i).suffix == '.pqr']

<a id="viewPockets"></a>
### Visualizing selected pockets (cavities)
Visualizing the selected **pockets** (cavities) from the filtered list using **NGL viewer**.<br>

**Protein residues** forming the **cavity** are represented in **random-colored surfaces**. **Pockets** are represented in a **blue-colored mesh**. Different **pockets** are identified with a floating **label**.

In [29]:
import re
import random

# random colors for cavities
r = lambda: random.randint(0,255)

# load structure
view = nglview.NGLWidget()
c = view.add_component(pdb_protein)

# load cavities (d) and pockets (p) and create pocketNames list
c = {}
p = {}
pocketNames = []
for pock in path_pockets:
    g = re.findall('(?:pocket)(\d+)(?:_\w+)\.(\w+)', pock)
    i = g[0][0]
    suff = g[0][1]
    if not [item for item in pocketNames if ('pocket' + i) in item]: pocketNames.append(('pocket' + i, int(i)))

    if suff == 'pdb':
        c[i] = view.add_component(pock)
        c[i].clear()
    else:
        p[i] = view.add_component(pock)
        p[i].clear()

# sort pocket names
pocketNames.sort(key=lambda tup: tup[1])
        
# representation for cavities
for pock in path_pockets_pdb:
    g = re.findall('(?:pocket)(\d+)(?:_\w+)\.(\w+)', pock)
    i = g[0][0]
    c[i].add_surface(color='#%02X%02X%02X' % (r(),r(),r()),  
                     radius='1.5',
                     lowResolution= True,
                     # 0: low resolution 
                     smooth=1,
                     useWorker= True,
                     wrap= True)
    
# representation for pockets
for pock in path_pockets_pqr:
    g = re.findall('(?:pocket)(\d+)(?:_\w+)\.(\w+)', pock)
    i = g[0][0]
    p[i].add_surface( component=i, color='skyblue', surfaceType= 'av', contour=True )

view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

# show pocket labels
code = """
var stage = this.stage;
var view = this.stage.viewer;
var clist_len = stage.compList.length;
var i = 0;
for(i = 0; i <= clist_len; i++){
    if(stage.compList[i] != undefined && stage.compList[i].structure != undefined && stage.compList[i].object.name.indexOf('pqr') != -1) {        

        var elm = document.createElement("div");
        elm.innerText = 'pocket' + stage.compList[i].object.name.match(/\d+/g)
        elm.style.color = "black";
        elm.style.background = "rgba(201, 149, 6, .8)";
        elm.style.padding = "8px";
        
        stage.compList[i].addAnnotation(stage.compList[i].structure.center, elm)
    }
}
"""

view._execute_js_code(code)

view.render_image()
view.download_image(filename='ngl3.png')
view

NGLWidget()

<img src='ngl3.png'></img>

<a id="selectPockets"></a>
### Select pocket (cavity)
Select a specific **pocket** (cavity) from the filtered list to be used in the **docking procedure**. <br>

If **fpocket** has been able to identify the correct **binding site**, which we know from the original **protein-ligand structure**, it just needs to be selected. In this particular example, the pocket we are interested in is the **pocket number 6**. <br>

Choose a **pocket** from the **DropDown list**:

In [11]:
mdsel = ipywidgets.Dropdown(
    options=pocketNames,
    description='Sel. pocket:',
    disabled=False,
)
display(mdsel)

Dropdown(description='Sel. pocket:', options=(('pocket1', 1), ('pocket6', 6)), value=1)

<a id="fpocketSelect"></a>
***
## Extract Pocket Cavity 
Extract the selected **protein cavity** (pocket) from the **fpocket** results.<br>

It will be used to generate the **docking box** in the **following step**.

***
**Building Blocks** used:
 - [fpocket_select](https://biobb-vs.readthedocs.io/en/latest/fpocket.html#module-fpocket.fpocket_select) from **biobb_vs.fpocket.fpocket_select**
***

In [12]:
from biobb_vs.fpocket.fpocket_select import fpocket_select

fpocket_cavity = "fpocket_cavity.pdb"
fpocket_pocket = "fpocket_pocket.pqr"
prop = {
    "pocket": mdsel.value
}

fpocket_select(input_pockets_zip=fpocket_filter_pockets,
                output_pocket_pdb = fpocket_cavity,
                output_pocket_pqr=fpocket_pocket,
                properties=prop)

2021-05-17 15:45:08,171 [MainThread  ] [INFO ]  Creating 7d17de5e-802f-45b7-8830-92f5b35506d7 temporary folder
2021-05-17 15:45:08,174 [MainThread  ] [INFO ]  Extracting: /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_virtual-screening/biobb_wf_virtual-screening/notebooks/fpocket/fpocket_filter_pockets.zip
2021-05-17 15:45:08,175 [MainThread  ] [INFO ]  to:
2021-05-17 15:45:08,176 [MainThread  ] [INFO ]  ['7d17de5e-802f-45b7-8830-92f5b35506d7/pocket1_atm.pdb', '7d17de5e-802f-45b7-8830-92f5b35506d7/pocket1_vert.pqr', '7d17de5e-802f-45b7-8830-92f5b35506d7/pocket6_atm.pdb', '7d17de5e-802f-45b7-8830-92f5b35506d7/pocket6_vert.pqr']
2021-05-17 15:45:08,178 [MainThread  ] [INFO ]  Saving fpocket_cavity.pdb file
2021-05-17 15:45:08,179 [MainThread  ] [INFO ]  Saving fpocket_pocket.pqr file
2021-05-17 15:45:08,181 [MainThread  ] [INFO ]  Removed temporary folder: 7d17de5e-802f-45b7-8830-92f5b35506d7


0

<a id="cavityBox"></a>
***
## Generating Cavity Box 
Generating a **box** surrounding the selected **protein cavity** (pocket), to be used in the **docking procedure**. The **box** is defining the region on the **surface** of the **protein target** where the **docking program** should explore a possible **ligand dock**.<br>
An offset of **12 Angstroms** is used to generate a **big enough box** to fit the **small molecule** and its possible rotations.<br>

***
**Building Blocks** used:
 - [box](https://biobb-vs.readthedocs.io/en/latest/utils.html#module-utils.box) from **biobb_vs.utils.box**
***

In [13]:
from biobb_vs.utils.box import box

output_box = "box.pdb"
prop = {
    "offset": 12,
    "box_coordinates": True
}

box(input_pdb_path = fpocket_pocket,
            output_pdb_path = output_box,
            properties=prop)

/anaconda3/envs/biobb_VS_tutorial/lib/python3.7/site-packages/Bio/SubsMat/__init__.py:131: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  BiopythonDeprecationWarning,
2021-05-17 15:45:10,804 [MainThread  ] [INFO ]  Loading pocket PQR selection from fpocket_pocket.pqr
2021-05-17 15:45:10,807 [MainThread  ] [INFO ]  Binding site center (Angstroms):     18.588    -3.586   -17.901
2021-05-17 15:45:10,808 [MainThread  ] [INFO ]  Adding 12.0 Angstroms offset
2021-05-17 15:45:10,809 [MainThread  ] [INFO ]  Binding site size (Angstroms):       18.884    25.442    23.006
2021-05-17 15:45:10,811 [MainThread  ] [INFO ]  Volume (cubic Angstroms): 88425
2021-05-17 15:45:10,812 [MainThread  ] [INFO ]  Adding box coordinates
2021-05-17 15:45:10,814 [MainThr

0

<a id="vis3D"></a>
### Visualizing binding site box in 3D structure
Visualizing the **protein structure**, the **selected cavity**, and the **generated box**, all together using **NGL** viewer. Using the **original structure** with the **small ligand** inside (Imatinib, [STI](https://www.rcsb.org/ligand/STI)), to check that the **selected cavity** is placed in the **same region** as the **original ligand**. 

In [30]:
view = nglview.NGLWidget()
s = view.add_component(download_pdb)
b = view.add_component(output_box)
p = view.add_component(fpocket_pocket)
p.clear()

atomPair = [
    [ "9999:Z.ZN1", "9999:Z.ZN2" ],
    [ "9999:Z.ZN2", "9999:Z.ZN4" ],
    [ "9999:Z.ZN4", "9999:Z.ZN3" ],
    [ "9999:Z.ZN3", "9999:Z.ZN1" ],
    
    [ "9999:Z.ZN5", "9999:Z.ZN6" ],
    [ "9999:Z.ZN6", "9999:Z.ZN8" ],
    [ "9999:Z.ZN8", "9999:Z.ZN7" ],
    [ "9999:Z.ZN7", "9999:Z.ZN5" ],
    
    [ "9999:Z.ZN1", "9999:Z.ZN5" ],
    [ "9999:Z.ZN2", "9999:Z.ZN6" ],
    [ "9999:Z.ZN3", "9999:Z.ZN7" ],
    [ "9999:Z.ZN4", "9999:Z.ZN8" ]
]

# structure
s.add_representation(repr_type='cartoon', 
                        selection='not het',
                        color='#cccccc',
                       opacity=.2)
# ligands box
b.add_representation(repr_type='ball+stick',
                     selection='9999',
                     color='pink', 
                     aspectRatio = 8)
# lines box
b.add_representation(repr_type='distance', 
                     atomPair= atomPair,
                     labelVisible=False,
                     color= 'black')

# pocket
p.add_surface(component=mdsel.value, 
              color='skyblue', 
              surfaceType= 'av', 
              lowResolution= True,
              # 0: low resolution 
              smooth=1,
              contour=True,
              opacity=0.4,
              useWorker= True,
              wrap= True )


view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])

view.render_image()
view.download_image(filename='ngl4.png')
view

NGLWidget()

<img src='ngl4.png'></img>

<a id="downloadSmallMolecule"></a>
***
## Downloading Small Molecule 
Downloading the desired **small molecule** to be used in the **docking procedure**. <br>
In this particular example, the small molecule of interest is the FDA-approved drug **Imatinib**, with PDB code **STI**.<br>

***
**Building Blocks** used:
 - [ideal_sdf](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.ideal_sdf) from **biobb_io.api.ideal_sdf**
***

In [15]:
from biobb_io.api.ideal_sdf import ideal_sdf

sdf_ideal = "ideal.sdf"
prop = {
  "ligand_code": ligand_code
}

ideal_sdf(output_sdf_path=sdf_ideal,
    properties=prop)


2021-05-17 15:45:17,292 [MainThread  ] [INFO ]  Downloading: STI from: ftp://ftp.ebi.ac.uk/pub/databases/msd/pdbechem/files/sdf/STI.sdf
2021-05-17 15:45:17,294 [MainThread  ] [INFO ]  Writting sdf to: ideal.sdf


0

<a id="sdf2pdb"></a>
***
## Converting Small Molecule 
Converting the desired **small molecule** to be used in the **docking procedure**, from **SDF** format to **PDB** format using the **OpenBabel chemoinformatics** tool. <br>

***
**Building Blocks** used:
 - [babel_convert](https://biobb-chemistry.readthedocs.io/en/latest/babelm.html#module-babelm.babel_convert) from **biobb_chemistry.babelm.babel_convert**
***

In [16]:
from biobb_chemistry.babelm.babel_convert import babel_convert

ligand = "ligand.pdb"
prop = {
    "input_format": "sdf",
    "output_format": "pdb",
    "obabel_path": "obabel"
}

babel_convert(input_path = sdf_ideal,
            output_path = ligand,
            properties=prop)

2021-05-17 15:45:19,010 [MainThread  ] [INFO ]  Value  is not compatible as a coordinates value
2021-05-17 15:45:19,011 [MainThread  ] [INFO ]  Not using any container
2021-05-17 15:45:19,129 [MainThread  ] [INFO ]  obabel -isdf ideal.sdf -opdb -Oligand.pdb  

2021-05-17 15:45:19,131 [MainThread  ] [INFO ]  Exit code 0

2021-05-17 15:45:19,131 [MainThread  ] [INFO ]  1 molecule converted



0

<a id="ligand_pdb2pdbqt"></a>
***
## Preparing Small Molecule (ligand) for Docking
Preparing the **small molecule** structure for the **docking procedure**. Converting the **PDB file** to a **PDBQT file** format (AutoDock PDBQT: Protein Data Bank, with Partial Charges (Q), & Atom Types (T)), needed by **AutoDock Vina**. <br><br>
The process adds **partial charges** and **atom types** to every atom. Besides, the **ligand flexibility** is also defined in the information contained in the file. The concept of **"torsion tree"** is used to represent the **rigid and rotatable** pieces of the **ligand**. A rigid piece (**"root"**) is defined, with zero or more rotatable pieces (**"branches"**), hanging from the root, and defining the **rotatable bonds**.<br><br>
More info about **PDBQT file format** can be found in the [AutoDock FAQ pages](http://autodock.scripps.edu/faqs-help/faq/what-is-the-format-of-a-pdbqt-file).

***
**Building Blocks** used:
 - [babel_convert](https://biobb-chemistry.readthedocs.io/en/latest/babelm.html#module-babelm.babel_convert) from **biobb_chemistry.babelm.babel_convert**
***

In [17]:
from biobb_chemistry.babelm.babel_convert import babel_convert

prep_ligand = "prep_ligand.pdbqt"
prop = {
    "input_format": "pdb",
    "output_format": "pdbqt",
    "obabel_path": "obabel"
}

babel_convert(input_path = ligand,
            output_path = prep_ligand,
            properties=prop)

2021-05-17 15:45:21,318 [MainThread  ] [INFO ]  Value  is not compatible as a coordinates value
2021-05-17 15:45:21,321 [MainThread  ] [INFO ]  Not using any container
2021-05-17 15:45:21,399 [MainThread  ] [INFO ]  obabel -ipdb ligand.pdb -opdbqt -Oprep_ligand.pdbqt  

2021-05-17 15:45:21,400 [MainThread  ] [INFO ]  Exit code 0

2021-05-17 15:45:21,401 [MainThread  ] [INFO ]  1 molecule converted



0

<a id="viewDrug"></a>
### Visualizing small molecule (drug)
Visualizing the desired **drug** to be docked to the **target protein**, using **NGL viewer**.<br>
- **Left panel**: **PDB-formatted** file, with all hydrogen atoms.
- **Right panel**: **PDBqt-formatted** file (AutoDock Vina-compatible), with **united atom model** (only polar hydrogens are placed in the structures to correctly type heavy atoms as hydrogen bond donors).


In [31]:
from ipywidgets import HBox

v0 = nglview.show_structure_file(ligand)
v1 = nglview.show_structure_file(prep_ligand)

v0._set_size('500px', '')
v1._set_size('500px', '')

def on_change(change):
    v1._set_camera_orientation(change['new'])
    
v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])

<img src='ngl5.png'></img>

<a id="protein_pdb2pdbqt"></a>
***
## Preparing Target Protein for Docking
Preparing the **target protein** structure for the **docking procedure**. Converting the **PDB file** to a **PDBqt file**, needed by **AutoDock Vina**. Similarly to the previous step, the process adds **partial charges** and **atom types** to every target protein atom. In this case, however, we are not taking into account **receptor flexibility**, although **Autodock Vina** allows some limited flexibility of selected **receptor side chains** [(see the documentation)](https://autodock-vina.readthedocs.io/en/latest/docking_flexible.html).<br>

***
**Building Blocks** used:
 - [str_check_add_hydrogens](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#utils-str-check-add-hydrogens-module) from **biobb_structure_utils.utils.str_check_add_hydrogens**
***

In [19]:
from biobb_structure_utils.utils.str_check_add_hydrogens import str_check_add_hydrogens

prep_receptor = "prep_receptor.pdbqt"
prop = {
    "charges": True,
    "mode": "auto"
}

str_check_add_hydrogens(input_structure_path = pdb_protein,
            output_structure_path = prep_receptor,
            properties=prop)

2021-05-17 15:45:26,940 [MainThread  ] [INFO ]  check_structure -i /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_virtual-screening/biobb_wf_virtual-screening/notebooks/fpocket/pdb_protein.pdb -o prep_receptor.pdbqt --force_save add_hydrogen --add_charges --add_mode auto

2021-05-17 15:45:26,942 [MainThread  ] [INFO ]  Exit code 0

2021-05-17 15:45:26,943 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.7.2                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-20                  =

Structure /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_virtual-screening/biobb_wf_virtual-screening/notebooks/fpocket/pdb_protein.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution: None A

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  329
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified resid

0

<a id="docking"></a>
***
## Running the Docking
Running the **docking process** with the prepared files:
- **ligand**
- **target protein**
- **binding site box**<br>

using **AutoDock Vina**. <br>

***
**Building Blocks** used:
 - [autodock_vina](https://biobb-vs.readthedocs.io/en/latest/vina.html#module-vina.autodock_vina) from **biobb_vs.vina.autodock_vina**
***

In [20]:
from biobb_vs.vina.autodock_vina import autodock_vina

output_vina_pdbqt = "output_vina.pdbqt"
output_vina_log = "output_vina.log"

autodock_vina(input_ligand_pdbqt_path = prep_ligand,
             input_receptor_pdbqt_path = prep_receptor,
             input_box_path = output_box,
             output_pdbqt_path = output_vina_pdbqt,
             output_log_path = output_vina_log)

2021-05-17 15:45:29,614 [MainThread  ] [INFO ]  prep_receptor.pdbqt file ends with END, cleaning
2021-05-17 15:45:29,618 [MainThread  ] [INFO ]  Executing AutoDock Vina
2021-05-17 15:45:29,619 [MainThread  ] [INFO ]  Not using any container
2021-05-17 15:46:19,009 [MainThread  ] [INFO ]  vina --ligand prep_ligand.pdbqt --receptor prep_receptor.pdbqt --center_x=18.588 --center_y=-3.586 --center_z=-17.901 --size_x=18.884 --size_y=25.442 --size_z=23.006 --out output_vina.pdbqt --log output_vina.log

2021-05-17 15:46:19,011 [MainThread  ] [INFO ]  Exit code 0

2021-05-17 15:46:19,011 [MainThread  ] [INFO ]  #################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and  

0

<a id="viewDocking"></a>
### Visualizing docking output poses
Visualizing the generated **docking poses** for the **ligand**, using **NGL viewer**. <br>
- **Left panel**: **Docking poses** displayed with atoms coloured by **partial charges** and **licorice** representation.
- **Right panel**: **Docking poses** displayed with atoms coloured by **element** and **ball-and-stick** representation.

In [21]:
from ipywidgets import HBox

models = 'all'

v0 = nglview.show_structure_file(output_vina_pdbqt, default=False)
v0.add_representation(repr_type='licorice', 
                        selection=models,
                       colorScheme= 'partialCharge')
v0.center()
v1 = nglview.show_structure_file(output_vina_pdbqt, default=False)
v1.add_representation(repr_type='ball+stick', 
                        selection=models)
v1.center()

v0._set_size('500px', '')
v1._set_size('500px', '')

def on_change(change):
    v1._set_camera_orientation(change['new'])
    
v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])

<img src='ngl6.png'></img>

<a id="selectPose"></a>
### Select Docking Pose
Select a specific **docking pose** from the output list for **visual inspection**.
<br>
Choose a **docking pose** from the **DropDown list**.

In [22]:
from Bio.PDB import PDBParser
parser = PDBParser(QUIET = True)
structure = parser.get_structure("protein", output_vina_pdbqt)
models = []
for i, m in enumerate(structure):
    models.append(('model' + str(i), i))
    
mdsel = ipywidgets.Dropdown(
    options=models,
    description='Sel. model:',
    disabled=False,
)
display(mdsel)

Dropdown(description='Sel. model:', options=(('model0', 0), ('model1', 1), ('model2', 2), ('model3', 3), ('mod…

<a id="extractPose"></a>
***
## Extract a Docking Pose
Extract a specific **docking pose** from the **docking** outputs. <br>

***
**Building Blocks** used:
 - [extract_model_pdbqt](https://biobb-vs.readthedocs.io/en/latest/utils.html#module-utils.extract_model_pdbqt) from **biobb_vs.utils.extract_model_pdbqt**
***

In [23]:
from biobb_vs.utils.extract_model_pdbqt import extract_model_pdbqt

output_pdbqt_model = "output_model.pdbqt"
prop = {
    "model": mdsel.value + 1
}

extract_model_pdbqt(input_pdbqt_path = output_vina_pdbqt,
             output_pdbqt_path = output_pdbqt_model,
            properties=prop)

2021-05-17 15:46:56,618 [MainThread  ] [INFO ]  Saving model 1 to output_model.pdbqt


0

<a id="pdbqt2pdb"></a>
***
## Converting Ligand Pose to PDB format
Converting **ligand pose** to **PDB format**. <br>

***
**Building Blocks** used:
 - [babel_convert](https://biobb-chemistry.readthedocs.io/en/latest/babelm.html#module-babelm.babel_convert) from **biobb_chemistry.babelm.babel_convert**
***

In [24]:
from biobb_chemistry.babelm.babel_convert import babel_convert

output_pdb_model = "output_model.pdb"
prop = {
    "input_format": "pdbqt",
    "output_format": "pdb",
    "obabel_path": "obabel"
}

babel_convert(input_path = output_pdbqt_model,
             output_path = output_pdb_model,
            properties=prop)

2021-05-17 15:46:58,600 [MainThread  ] [INFO ]  Value  is not compatible as a coordinates value
2021-05-17 15:46:58,602 [MainThread  ] [INFO ]  Not using any container
2021-05-17 15:46:58,658 [MainThread  ] [INFO ]  obabel -ipdbqt output_model.pdbqt -opdb -Ooutput_model.pdb  

2021-05-17 15:46:58,659 [MainThread  ] [INFO ]  Exit code 0

2021-05-17 15:46:58,660 [MainThread  ] [INFO ]  1 molecule converted



0

<a id="catPdb"></a>
***
## Superposing Ligand Pose to the Target Protein Structure
Superposing **ligand pose** to the target **protein structure**, in order to see the **protein-ligand docking conformation**. <br><br>Building a new **PDB file** with both **target and ligand** (binding pose) structures. <br>

***
**Building Blocks** used:
 - [cat_pdb](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.cat_pdb) from **biobb_structure_utils.utils.cat_pdb**
***

In [25]:
from biobb_structure_utils.utils.cat_pdb import cat_pdb

output_structure = "output_structure.pdb"

cat_pdb(input_structure1 = pdb_protein,
             input_structure2 = output_pdb_model,
             output_structure_path = output_structure)

2021-05-17 15:47:00,632 [MainThread  ] [INFO ]  File output_structure.pdb created


0

<a id="viewFinal"></a>
### Comparing final result with experimental structure 
Visualizing and comparing the generated **protein-ligand** complex with the original **protein-ligand conformation** (downloaded from the PDB database), using **NGL viewer**. <br>
- **Licorice, element-colored** representation: **Experimental pose**.
- **Licorice, green-colored** representation: **Docking pose**.
<br>

Note that outputs from **AutoDock Vina** don't contain all the atoms, as the program works with a **united-atom representation** (i.e. only polar hydrogens).

In [32]:
view = nglview.NGLWidget()

# v1 = Experimental Structure
v1 = view.add_component(download_pdb)

v1.clear()
v1.add_representation(repr_type='licorice', 
                     selection='STI',
                     radius=0.5)

# v2 = Docking result
v2 = view.add_component(output_structure)
v2.clear()
v2.add_representation(repr_type='cartoon', colorScheme = 'sstruc')
v2.add_representation(repr_type='licorice', radius=0.5, color= 'green', selection='UNL')

view._remote_call('setSize', target='Widget', args=['','600px'])
view

# align reference and output
code = """
var stage = this.stage;
var clist_len = stage.compList.length;
var i = 0;
var s = [];
for(i = 0; i <= clist_len; i++){
    if(stage.compList[i] != undefined && stage.compList[i].structure != undefined) {        
       s.push(stage.compList[i])
    }
}
NGL.superpose(s[0].structure, s[1].structure, true, ".CA")
s[ 0 ].updateRepresentations({ position: true })
s[ 0 ].autoView()
"""

view._execute_js_code(code)

view.render_image()
view.download_image(filename='ngl7.png')
view

NGLWidget()

<img src='ngl7.png'></img>

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)
